### Building Question answering system

For  this work , we will build our question answering system , we will leverage the [deepset framework]() to build the components of our system.

Here are the followign components of our system:
-  The index store 
- The document store 
- The search pipeline with a Retrieval and a Reader model.

We will be leveraging the tutorials provided by deepstack to build our system.

For this work we will leverage [this  tutorial:](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial6_Better_Retrieval_via_DPR.ipynb) 

### Building the dense passage retrieval

To build the dataset for the dense passage retrieval , we will be using the approach suggested by [this tutorial](https://huggingface.co/etalab-ia/dpr-question_encoder-fr_qa-camembert) which use the cambert model .

For each question , we have a single positive context , the paragraph where the answer to the question is located and n hard negavives contexts that are the the top - k canditates taht does not contain the answer, to the question. to retrieve the negative context they will use the bm25 retrieval model.

Before training the retrieval model we will have to build the document store and save the document as units of retrieval to the store.

### Building the dataset

In [1]:
import pandas as pd
import numpy as np

In [3]:
from pathlib import Path
DATA_PATH = Path.cwd().joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"
TEXT_DATA_FOLDER = DATA_PATH.joinpath("corpus", "drc-news-txt")
assert TEXT_DATA_FOLDER.exists(), "the text data folder does not exist"

In [4]:
data_file_path = DATA_PATH.joinpath("corpus", "raw", 'drc-news-raws.csv')

In [5]:
data = pd.read_csv(data_file_path, names=["content", "posted_at"])

In [6]:
data.shape

(140638, 2)

In [7]:
data = data.fillna(value="")
data.head()

,content,posted_at
0,Les membres de la Commission tarifaire viennen...,2022-09-05 00:00:00
1,Les membres de la Commission tarifaire so...,2022-04-05 00:00:00
2,Vodacom Congo vient de signer un partenariat a...,2022-04-23 00:00:00
3,"Le sélectionneur des Léopards de la RDC, Hectó...",2022-03-05 00:00:00
4,Le protocole d’accord était déjà signé entre l...,2022-11-05 00:00:00


In [8]:
from haystack.nodes import TextConverter

/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
WARNING - haystack -  Object '__file__' is imported through a deprecated path. Please check out the docs for the new import path.
WARNING - haystack -  Object '__file__' is imported through a deprecated path. Please check out the docs for the new import path.
WARNING - haystack -  Object '__file__' is imported through a

In [11]:
from haystack.schema import Document
from secrets import token_hex

# @Todo: this is not working now , it was supposed to save the document to dataframe
def get_document_from_text(row):
    """numpy row with the text and the date of the post

    Args:
        row (_type_): _description_

    Returns:
        _type_: _description_
    """
    text = row[0].replace(u'\xa0', u' ')
    for paragraph in text.split("   "):
        if not paragraph.strip():  # skip empty paragraphs
            continue
        return Document(content=paragraph, meta={"posted_at":row[1] if row[1] else "" })

In [12]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.utils import convert_files_to_docs

In [1]:

# in the final version , split in  a training and validation set.

In [13]:
all_docs = data.sample(200).apply(get_document_from_text, axis="columns")

In [14]:
all_docs.shape

(200,)

In [16]:
all_docs = all_docs.dropna().to_list()

AttributeError: 'list' object has no attribute 'dropna'

In [17]:
from haystack.errors import HaystackError
from haystack.schema import Document
from typing import List, Optional, Generator, Set, Union
from copy import deepcopy
from haystack.nodes import PreProcessor

class CustomPreProcessor(PreProcessor):
    def __init__(self, custom_preprocessor=None, **kwargs):
        super().__init__(**kwargs)
        self.custom_preprocessor = custom_preprocessor
    def clean(
        self,
        document: Union[dict, Document],
        clean_whitespace: bool,
        clean_header_footer: bool,
        clean_empty_lines: bool,
        remove_substrings: List[str],
        id_hash_keys: Optional[List[str]] = None,
    ) -> Document:
        """
        
        Perform document cleaning on a single document and return a single document. This method will deal with whitespaces, headers, footers
        and empty lines. Its exact functionality is defined by the parameters passed into PreProcessor.__init__().
        """
        if id_hash_keys is None:
            id_hash_keys = self.id_hash_keys

        if isinstance(document, dict):
            document = Document.from_dict(document, id_hash_keys=id_hash_keys)

        # Mainly needed for type checking
        if not isinstance(document, Document):
            raise HaystackError("Document must not be of type 'dict' but of type 'Document'.")
        text = document.content
        text = self.custom_preprocessor(text)
        if clean_header_footer:
            text = self._find_and_remove_header_footer(
                text, n_chars=300, n_first_pages_to_ignore=1, n_last_pages_to_ignore=1
            )

        if clean_whitespace:
            lines = text.splitlines()

            cleaned_lines = []
            for line in lines:
                line = line.strip()
                cleaned_lines.append(line)
            text = "\n".join(cleaned_lines)

        if clean_empty_lines:
            text = re.sub(r"\n\n+", "\n\n", text)

        for substring in remove_substrings:
            text = text.replace(substring, "")

        if text != document.content:
            document = deepcopy(document)
            document.content = text

        return document
    
    

In [18]:
import re
from gensim.utils import deaccent
from unicodedata import normalize as unicode_normalize

In [19]:
def replace_point(document):
    """replace the point with the wwt.www with space point before tokenizing the document .
    TOdos : this may have a a downside when the point is in the middle of a words
    Args:
        document (_type_): _description_
    """
    result = re.sub(r"(\S)\.(\S)", r"\1 . \2", document)
    return result

def replace_website_name(document):
    """sometimes the doucment has the name politico.cd or 7sur7.cd or actualite.cd, we would like to replace them by the 
    actual name of the website. before proper cleaning

    Args:
        document (_type_): _description_
    """
    # @TODO : not sure if this will work but , way better replace by the first line of match.
    
    result = re.sub(r"7SUR7.CD|politico.cd|actualite.cd|mediacongo.net", r"SITE_WEB", document, flags=re.IGNORECASE)
    return result

def remove_accents(document):
    input_without_accent = deaccent(document)
    return input_without_accent

def pre_clean_document(document):
    """pre clean the document by removing the accents and replacing the point with the wwt.www with space point before tokenizing the document .
    TOdos : this may have a a downside when the point is in the middle of a words
    and any other side of cleaning that we want to do .
    Args:
        document (_type_): _description_
    """
    result = remove_accents(document)
    result =  replace_website_name(result)
    result = replace_point(result)
    result = re.sub(r"This post has already been read \d+ times!", "", result) # remove unwanted text
    result = unicode_normalize("NFKD", result)
    return result

In [20]:
text_doc = """
Une motion de defiance a ete deposee au cabinet de la presidente de l'Assemblee provinciale du Maniema contre le vice-gouverneur Jean-Pierre Amadi, le mardi 30 mars dernier.16 parmi les 17 deputes provinciaux presents a Kindu, chef-lieu de la province du Maniema, ont appose leurs signatures sur ladite motion depuis le 27 mars 2021.Selon ce document consulte par 7SUR7.CD, Jean-Pierre Amadi Lubenga est reproche de plusieurs griefs dont le « refus d'obtemperer aux instructions de la hierarchie » pendant qu'il etait gouverneur de province a l'interim et le detournement des deniers publics.
comme signale a politico.cd sur notre site POLITICO.CD et puis ensuite sur actualite.cd et sur notre site mediacongo.net
"""

In [21]:
replace_website_name(text_doc)

"\nUne motion de defiance a ete deposee au cabinet de la presidente de l'Assemblee provinciale du Maniema contre le vice-gouverneur Jean-Pierre Amadi, le mardi 30 mars dernier.16 parmi les 17 deputes provinciaux presents a Kindu, chef-lieu de la province du Maniema, ont appose leurs signatures sur ladite motion depuis le 27 mars 2021.Selon ce document consulte par SITE_WEB, Jean-Pierre Amadi Lubenga est reproche de plusieurs griefs dont le « refus d'obtemperer aux instructions de la hierarchie » pendant qu'il etait gouverneur de province a l'interim et le detournement des deniers publics.\ncomme signale a SITE_WEB sur notre site SITE_WEB et puis ensuite sur SITE_WEB et sur notre site SITE_WEB\n"

In [22]:
preprocessor = CustomPreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    language="fr",
    custom_preprocessor=pre_clean_document,
)


docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

100%|██████████| 200/200 [00:00<00:00, 438.67docs/s]

n_files_input: 200
n_docs_output: 491


In [23]:
docs[0]

<Document: {'content': '« Je lui (Sophie Wilmes) ai dit que le conflit entre Kabila et Tshisekedi c’est pour distraire et les congolais et la communaute internationale parce qu’il y avait 3 crises majeures : la crise de legitimite, la crise sociale et la crise budgetaire et la crise sur les menaces de balkanisation. Des querelles qui vont dans le sens de dire que la coalition ne marche pas parce que les militants ne savent pas exactement ce que monsieur Kabila et monsieur Tshisekedi ont signe comme deal et nous pensons que pour nous tout ca c’est du mascara. C’est quelque chose bien entretenue entre Tshisekedi et Kabila et qu’ils cachent aux militants, pour eux ils sont besoin de l’un de l’autre pour continuer leur deal. Nous avons dit que ceux qui pensent que Tshisekedi et Kabila vont se separer, ils ont tort. Ces deux-la vont marcher a l’equilibre instable provoquant des crises pour que le peuple ne se dessaisisse des questions brulantes aujourd’hui », a dit a SITE_WEB Martin Fayulu.

In [24]:
from haystack.document_stores import ElasticsearchDocumentStore



In [25]:
document_store = ElasticsearchDocumentStore(index="drc-news", recreate_index=True, analyzer="french")

/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
INFO - haystack.document_stores.elasticsearch -  Index 'drc-news' deleted.
INFO - haystack.document_stores.elasticsearch -  Index 'label' deleted.


In [26]:
document_store.write_documents(docs)

### Retrieval

With the document store in place , the document store has all the document in it , let build a retriever model that use BM25 to retrieve the document.

In [27]:
custom_query_template = """
{
  "query": {
    "boosting": {
      "positive": {
        "match": {
          "content": ${query}
        }
      },
      "negative": {
        "match": {
          "content": ${name_to_not_match}
        }
      },
      "negative_boost": 0.5
    }
  }
}
"""

In [28]:
from haystack.nodes import BM25Retriever

In [29]:
bm25_retriever = BM25Retriever(document_store=document_store, all_terms_must_match=True, custom_query=custom_query_template)
bm25_retriever_positive = BM25Retriever(document_store=document_store, all_terms_must_match=True)

In [30]:
question = "le president de la Republique democratique du congo?"
answer = "Felix Tshisekedi"

In [31]:
def get_hard_negative_context(
    retriever: BM25Retriever, question: str, answer: str, n_ctxs: int = 10
):
    """
    given the question and the answer query the Elastic search document store and return the hard negative context to the question
    """

    documents = bm25_retriever.retrieve(query=question, top_k=10, filters={"name_to_not_match": answer})
    return documents

In [32]:
get_hard_negative_context(bm25_retriever, question, answer)

[<Document: {'content': 'L’Envoye special Said Djinnit et le Secretaire executif de la CIRGL, le professeur Ntumba Luaba ont ete recus en audience par le president Joseph Kabila le jeudi dernier a Kinshasa. Au menu des discussions, la situation dans la region et en RDC et la Conference sur les investissements du secteur prive dans la region des Grands Lacs . Said Djinnit, l’Envoye special du Secretaire general des Nations Unies pour la region des Grands Lacs, a ete recu le jeudi dernier, avec le Secretaire executif de la Conference Internationale sur la Region des Grands Lacs (CIRGL) par le president de la Republique Democratique du Congo, Joseph Kabila . L’entretien a porte, selon la Mission de l’Organisation des Nations unies pour la stabilisation en Republique democratique du Congo (MONUSCO), sur la situation dans la region, y compris les efforts en cours en vue de la neutralisation de toutes les forces negatives operant a l’Est de la RDC . L’Envoye special Djinnit a exprime le sout

Next is to build the dense passage retrieval dataset , for each sentence we will find the name entities and mask them and query the database to find hard negative.

### Building the Reader Dataset

Adding the documents to the retriever store , the next step will be to build the dense passage retrieval dataset.

We will consider each paragraph as the answer, and we will generate differents question in the paragraph by masking the name entities which yield to a better score.

Once we have a question and the paragraph answer , we will retrieve the negative context with the code we wrote above.

#### NER on the Text

In [33]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


# this model is good but it is not classifiying roles exactly., we need to improve that. confusing ministre and ministere
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")

In [34]:
from transformers import pipeline

In [35]:
transformer_ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [36]:
from tqdm import tqdm
import json

In [43]:
import spacy

spacy_pipeline = spacy.load("fr_dep_news_trf")



In [37]:

random_id = np.random.randint(0, len(docs))
sample_document = docs[random_id]

In [38]:
sample_document

<Document: {'content': 'Il cite notamment : la neutralisation de plusieurs centaines de miliciensla reddition de nombreux elements de ces groupes armes , la recuperation d’armes et des munitions par les forces armees de la Republique , le demantelement de plusieurs reseaux de trafics illicites d’armes et munitions, des minerais, et de ravitaillement des groupes armes dans divers produits ; la recuperation de nombreuses localites de l’Est du pays qui etaient jadis coupees par des rebelles ; la liberation de nombreux otages autrefois detenus par les groupes armes dont les ADF ; la reouverture de certains axes routiers importants qui etait jadis sous le controle des forces negatives ; la reduction sensible des incursions contre la population civile.', 'content_type': 'text', 'score': None, 'meta': {'posted_at': '', '_split_id': 1}, 'embedding': None, 'id': 'e8e3bc1ebb40be8804720542badbb57a'}>

In [40]:
from src.data.corpus_builder_utils import DocumentContext, AllCorpusBuilder, Sentence

In [44]:
sample_document_context = DocumentContext(sample_document.content, 
                                          spacy_pipeline=spacy_pipeline,
                                          ner_pipeline=transformer_ner_pipeline,
                                          bm25_retriever_positive=bm25_retriever_positive,)

/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [40]:
sample_document_context.generate_sentences()

In [41]:
BASE_QA_PATH = DATA_PATH.joinpath("processed", "DRC-News-UQA")
assert BASE_QA_PATH.exists()

In [45]:
corpus_builder = AllCorpusBuilder(
    ner_pipeline=spacy_pipeline,
    transformer_pipeline=transformer_ner_pipeline,
    retriever=bm25_retriever_positive,
    all_docs=all_docs,
    base_folder=BASE_QA_PATH,
    corpus_name="drc-news-uqa-small-dev.json"
)

In [46]:
corpus_builder.build_corpus()

generating dataset:   0%|          | 0/200 [00:00<?, ?it/s]/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
generating dataset: 100%|██████████| 200/200 [11:56<00:00,  3.58s/it]


In [47]:
corpus_builder.combine_all_file_to_one()

Wrote 773 records to /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/processed/DRC-News-UQA/drc-news-uqa-small-dev.json


We all the utilities function we can go 2 ways from this: 
- train a dense passage retriever
- train a reader , leveraging our BM25 model and using the paper from the [span selection pretraining.](https://github.com/IBM/span-selection-pretraining/blob/master/sspt/sspt_gen_async.py)

#### Implementation of the Training

Retriever details 

For the retriever we will use the BM25 to retrieve for each question the positive context and the negative context.

The postive context are the paragraphs where the asnwer is located and the hard negative context are the context that does not contain the answer.

We will use only positive answer to train our model, for the dense passage retrieval fine tunning we will use the positive and negative context.